In [ ]:
import pydicom
from pydicom.filereader import dcmread
import numpy as np
import pandas as pd
import os
import shutil
import sys


# Convert Dicom to Nifti files

In [ ]:
from pydicom.filereader import dcmread
import glob
from tqdm import tqdm
tqdm.pandas()

#Convert dicom folders into nifti images while preseving file structure


rt_dir = '../infantBrainAge' #Root directory
rd_dir = 'raw_dataset_all'  #Directory of raw data (in root)
sv_dir = 'nifti_all_v2'  #Directory to save (in root)

pd_file = ' ' #Dataframe file with dicom folder names for each modality in separate fields and the rt_dir in 'bases' 
              #e.g. Header:    'T1'  | 'T2'  |  'bases'
              #Note: 'bases' should be organized as $root_dir/$accesion_number/
              
grouped_master_file_df = pd.read_pickle(pd_file)

for _, row in tqdm(grouped_master_file_df.iterrows(), total=len(grouped_master_file_df)):
    
    this_acc = row.bases.split('/')[-1]
    print(this_acc)
    output_folder = os.path.join(rt_dir,sv_dir,this_acc)
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)
    

    for T1_file in row.T1:
        dicom_directory = os.path.join(rt_dir, rd_dir, this_acc, T1_file)
        output_file = 'T1' + '___' + T1_file
        !dcm2niix -z y -o {output_folder} -f {output_file} {dicom_directory}
    
    for T2_file in row.T2:
        dicom_directory = os.path.join(rt_dir, rd_dir, this_acc, T2_file)
        output_file = 'T2' + '___' + T2_file
        !dcm2niix -z y -o {output_folder} -f {output_file} {dicom_directory}
    
    print()
    

# Batch Skullstrip

In [ ]:
# This will not work if you're not using my version of David's code (made some small changes to multiple files 
# to allow for faster batch processing) at: 
#    /data/rauschecker1/infantBrainAge/dweiss044-multipurpose_unet-1a072dd4d244/

import pydicom
from pydicom.filereader import dcmread
import numpy as np
import pandas as pd
import os
import shutil
import sys
import random
import traceback
import math
import glob

#import skullstripping library

In [ ]:

#The directory in which images are stored
rt_dir = '../infantBrainAge'
data_dir =  os.path.join(rt_dir,'nifti_all_nih')

#Directory where outputs will be saved
sv_dir = os.path.join('../infantBrainAge','A DIRECTORY TO SAVE IN')
if not os.path.isdir(sv_dir):
    os.mkdir(sv_dir)

In [ ]:
# Dataframe with filepaths for each modality AND unique_id for each image to register

#Mapping from modality to dataframe column with filename
data_field_map = {'T1': 'mri_native_t1_nii_file', 'T2': 'mri_native_t2_nii_file'} 

In [ ]:

#Initialize skullstripping class here
#...

for mod in ['T1','T2']: #['T1','T2','ADC']:
    
    in_files = []
    subjectIDs = []
    for _, row in meta_file.iterrows():
        if not pd.isna( row[data_field_map[mod]] ):
            
            in_files.append( row[data_field_map[mod]] )
    
    print('\n\nProcessing predictions..\n')
    
    for iii in range(len(in_files)):
        image_in = in_files[iii]
        print(image_in)
        image_out = os.path.join(sv_dir,image_in.split('/')[-1].replace('.nii.gz','_skullstripped.nii.gz'))
        
        #Run prediction of image_in and save in image_out here
        #...